<a href="https://colab.research.google.com/github/basilll007/Particle_phy_simulation/blob/main/particle_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import plotly.graph_objects as go


In [2]:
# PINN with residual connections and deeper architecture
class ComplexPINN(nn.Module):
    def __init__(self, input_dim=6, output_dim=9):
        super().__init__()
        self.fc_in = nn.Linear(input_dim, 512)
        self.hidden = nn.ModuleList([
            nn.Sequential(
                nn.Linear(512, 512),
                nn.LayerNorm(512),
                nn.GELU()
            ) for _ in range(8)
        ])
        self.fc_out = nn.Linear(512, output_dim)

    def forward(self, x):
        x = F.gelu(self.fc_in(x))
        for layer in self.hidden:
            residual = x
            x = layer(x) + residual
        return self.fc_out(x)



In [3]:
# Physics loss for momentum conservation
def physics_loss(output, input_vec):
    initial_p = input_vec[:, :3] + input_vec[:, 3:6]
    final_p = output[:, :3] + output[:, 3:6] + output[:, 6:9]
    return F.mse_loss(final_p, initial_p)

In [4]:
# synthetic data
def generate_data(num_samples=1000, mode="2-way"):
    data = []
    for _ in range(num_samples):
        p1 = np.random.randn(3)
        p2 = -p1 + np.random.normal(0, 0.1, 3)  # almost equal and opposite
        input_vec = np.concatenate((p1, p2))
        if mode == "2-way":
            output_vec = np.random.randn(6)
            output_vec = np.concatenate((output_vec, -output_vec[:3] - output_vec[3:6]))
        elif mode == "3-way":
            output_vec = np.random.randn(9)
        data.append((input_vec, output_vec))
    return torch.tensor([d[0] for d in data], dtype=torch.float32), torch.tensor([d[1] for d in data], dtype=torch.float32)


In [5]:
# Training function
def train_model(model, X, Y, epochs=1000, batch_size=64, lambda_phys=0.2):
    dataset = TensorDataset(X, Y)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
    loss_fn = nn.MSELoss()
    for epoch in range(epochs):
        for xb, yb in loader:
            pred = model(xb)
            loss_data = loss_fn(pred, yb)
            loss_phys = physics_loss(pred, xb)
            loss = loss_data + lambda_phys * loss_phys
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Data Loss: {loss_data.item():.4f}, Physics Loss: {loss_phys.item():.4f}")


In [6]:
# Visualize in Plotly
def visualize_collision(model, X, sample_index=0, title="Particle Collision"):
    model.eval()
    with torch.no_grad():
        vec = model(X[sample_index].unsqueeze(0)).numpy().flatten()
        fig = go.Figure()
        colors = ['red', 'green', 'blue']
        labels = ['Final Particle 1', 'Final Particle 2', 'Final Particle 3']
        for i in range(3):
            fig.add_trace(go.Cone(
                x=[0], y=[0], z=[0],
                u=[vec[i*3]], v=[vec[i*3+1]], w=[vec[i*3+2]],
                sizemode="absolute",
                anchor="tail",
                colorscale=[[0, colors[i]], [1, colors[i]]],
                showscale=False,
                name=labels[i]
            ))
        fig.update_layout(
            scene=dict(
                xaxis=dict(range=[-3,3], title="x"),
                yaxis=dict(range=[-3,3], title="y"),
                zaxis=dict(range=[-3,3], title="z")
            ),
            title=title,
            showlegend=True,
            width=700,
            height=700
        )
        fig.show()



In [7]:
# Run training and visualization
model_2way = ComplexPINN()
X2, Y2 = generate_data(1000, mode="2-way")
train_model(model_2way, X2, Y2)
visualize_collision(model_2way, X2, title="2-Way Collision")


/tmp/ipython-input-456910709.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  return torch.tensor([d[0] for d in data], dtype=torch.float32), torch.tensor([d[1] for d in data], dtype=torch.float32)


Epoch 0, Data Loss: 2.3382, Physics Loss: 1.2701
Epoch 100, Data Loss: 1.1892, Physics Loss: 0.0095
Epoch 200, Data Loss: 0.3673, Physics Loss: 0.0195
Epoch 300, Data Loss: 0.0503, Physics Loss: 0.0037
Epoch 400, Data Loss: 0.0089, Physics Loss: 0.0019
Epoch 500, Data Loss: 0.0019, Physics Loss: 0.0017
Epoch 600, Data Loss: 0.0005, Physics Loss: 0.0009
Epoch 700, Data Loss: 0.0006, Physics Loss: 0.0014
Epoch 800, Data Loss: 0.0027, Physics Loss: 0.0013
Epoch 900, Data Loss: 0.0340, Physics Loss: 0.0027


In [8]:
model_3way = ComplexPINN()
X3, Y3 = generate_data(1000, mode="3-way")
train_model(model_3way, X3, Y3)
visualize_collision(model_3way, X3, title="3-Way Collision")


Epoch 0, Data Loss: 1.7499, Physics Loss: 0.2712
Epoch 100, Data Loss: 0.8691, Physics Loss: 0.0736
Epoch 200, Data Loss: 0.5395, Physics Loss: 0.2393
Epoch 300, Data Loss: 0.1776, Physics Loss: 0.4135
Epoch 400, Data Loss: 0.1466, Physics Loss: 0.3133
Epoch 500, Data Loss: 0.1446, Physics Loss: 0.3162
Epoch 600, Data Loss: 0.1268, Physics Loss: 0.3450
Epoch 700, Data Loss: 0.1914, Physics Loss: 0.3747
Epoch 800, Data Loss: 0.1479, Physics Loss: 0.4102
Epoch 900, Data Loss: 0.1409, Physics Loss: 0.3786
